In [ ]:
#still need to split the data

import numpy as np
import pandas as pd
import math

# allow plots to appear in the noteboo

# create a custom colormap
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
from bokeh.plotting import figure,show,output_notebook
from bokeh.models import Range1d
from bokeh.io import hplot, output_file
from sklearn.naive_bayes import MultinomialNB #naive-bayes

from sklearn.cross_validation import ShuffleSplit #seems like this is needed for random forests


from sklearn.svm import SVC

#ROC Curve + Confusion_Matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import classification_report, confusion_matrix


from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn import cross_validation
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split

from sklearn.decomposition import PCA #SVD

output_notebook()
%matplotlib inline


mash = pd.read_csv("OnlineNewsPopularity.csv") #remember that read_csv auto places read data into a DF

#noticed that the headers all had a whitespace in it, so needed to remove it.
mash.columns = mash.columns.str.strip()

mash['url_id'] = mash.index 

#let's move the url_id column from the end to the top.

cols = mash.columns.tolist()
cols

cols = cols[-1:] + cols[:-1]
cols

mash = mash[cols]

exc_list_labels = ['timedelta','kw_min_min', 'kw_max_min', 'kw_avg_min', 'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 
                   'kw_max_avg', 'kw_avg_avg', 'self_reference_min_shares', 'self_reference_max_shares',
                  'self_reference_avg_sharess', 'LDA_00', 'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'url_id']

mash.drop('n_non_stop_words', axis = 1)

for value in exc_list_labels:
    mash = mash.drop([value], axis = 1)


mash.info()

what_day_append = []

#We are going to do 'categorize each day with a value


#each article has multiple values

for value in range(len(mash)):
    
    if(mash.weekday_is_monday[value] == 1):
        what_day_append.append(1) #if monday, then 1
    
    elif(mash.weekday_is_tuesday[value] == 1 ):
        what_day_append.append(2) #if tuesday, then 2
        
    elif(mash.weekday_is_wednesday[value] == 1):
        what_day_append.append(3) #if wednesday, then 3
        
    elif(mash.weekday_is_thursday[value] == 1):
        what_day_append.append(4) #if thursday, then 4
        
    elif(mash.weekday_is_friday[value] == 1):
        what_day_append.append(5) #if friday, then 5
        
    elif(mash.weekday_is_saturday[value] == 1):
        what_day_append.append(6) #if saturday, then 6
        
    elif(mash.weekday_is_sunday[value] == 1):
        what_day_append.append(7) #if sunday, then 7
        
    elif(mash.is_weekend[value] == 1):
        what_day_append.append(8) #if it's a weekend, then 8
        
    else:
        what_day_append.append(0) #shouldn't be any 0 values
        
len(what_day_append)

#mash.info()

mash['day'] = what_day_append
mash.day.value_counts() #see that there are no 0's

mash_life = mash[mash.data_channel_is_lifestyle == 1]
mash_ent = mash[mash.data_channel_is_entertainment == 1]
mash_bus = mash[mash.data_channel_is_bus == 1]
mash_socmed = mash[mash.data_channel_is_socmed == 1]
mash_tech = mash[mash.data_channel_is_tech == 1]
mash_world = mash[mash.data_channel_is_world == 1]

x = len(mash_life)
x2 = len(mash_ent)
x3 = len(mash_bus)
x4 = len(mash_socmed)
x5 = len(mash_tech)
x6 = len(mash_world)

#print x + x2 + x3 + x4 + x5 + x6 #33510
#print len(mash) #39644

#we are missing ~ 6000 data points that are unlabeled. 

rng_of_na = []

for value in range(len(mash)):
    if (mash.data_channel_is_lifestyle[value] != 1):
        if(mash.data_channel_is_entertainment[value] != 1):
            if(mash.data_channel_is_bus[value] != 1):
                if(mash.data_channel_is_socmed[value] != 1):
                    if(mash.data_channel_is_tech[value] != 1):
                        if(mash.data_channel_is_world[value] != 1):
                            rng_of_na.append(value) #we will have all the index
                            

na = len(rng_of_na)
#print na
#print x + x2 + x3 + x4 + x5 + x6 + na #all blog posts are now accounted for. 

m_n_1 = mash[mash.data_channel_is_lifestyle != 1]
m_n_2 = m_n_1[m_n_1.data_channel_is_entertainment != 1]
m_n_3 = m_n_2[m_n_2.data_channel_is_bus != 1]
m_n_4 = m_n_3[m_n_3.data_channel_is_socmed !=1]
m_n_5 = m_n_4[m_n_4.data_channel_is_tech != 1]
mash_na = m_n_5[m_n_5.data_channel_is_world !=1]

len(mash_na)

#remove the data channel and weekday labels

columns_remove_channels = ['data_channel_is_lifestyle', 'data_channel_is_entertainment', 'data_channel_is_bus', 
                  'data_channel_is_socmed', 'data_channel_is_tech', 'data_channel_is_world']

columns_remove_days = ['weekday_is_monday', 'weekday_is_tuesday', 'weekday_is_wednesday', 'weekday_is_thursday', 
                 'weekday_is_friday', 'weekday_is_saturday', 'weekday_is_sunday', 'is_weekend']

for value, value2 in zip(columns_remove_channels, columns_remove_days):
    mash_life = mash_life.drop([value], axis = 1)
    mash_life = mash_life.drop([value2], axis = 1)
    
    mash_ent = mash_ent.drop([value], axis = 1)
    mash_ent = mash_ent.drop([value2], axis = 1)
    
    mash_bus = mash_bus.drop([value], axis = 1)
    mash_bus = mash_bus.drop([value2], axis = 1)
    
    mash_socmed = mash_socmed.drop([value], axis = 1)
    mash_socmed = mash_socmed.drop([value2], axis = 1)
    
    mash_tech = mash_tech.drop([value], axis = 1)
    mash_tech = mash_tech.drop([value2], axis = 1)
    
    mash_world = mash_world.drop([value], axis = 1)
    mash_world = mash_world.drop([value2], axis = 1)
    
    mash_na= mash_na.drop([value], axis = 1)
    mash_na = mash_na.drop([value2], axis = 1)
    
#mash = mash.drop(columns_remove_channels, axis = 1)
#mash = mash.drop(columns_remove_days, axis = 1)



# KNN Analysis

In [1]:
import statsmodels.api as sm
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score #this is the valuation score when we use KMeans
import matplotlib.pyplot as plt

mash_lists = [mash_life, mash_ent, mash_bus, mash_socmed, mash_tech, mash_world]  
name_of_lists = ['mash_life' , 'mash_ent', 'mash_bus', 'mash_socmed', 'mash_tech', 'mash_world'] 


km_list_of_cvs = [] #this will take in all the values of the cross_val_score for each dataset
km_list_of_cm = []
km_list_of_cr = []

km_list_of_centers = [] #this will take in all the values of the centroids
km_list_of_labels = [] #this will take in all the values of the labels
km_list_of_s_scores = []

std_scalar = StandardScaler() #standardization

count = 0 #used for the naming

## beginning of for-loop

for mash_testing in mash_lists:
    
    print "Below is analysis on %s dataset" % name_of_lists[count]
    
    features = mash_testing.drop(['shares', 'url'], axis=1)
    target = mash_testing['shares']
    
    target_sorted = sorted(target) #sorting it so we can get the top 75%
    
    target_len = len(target_sorted)
    target_len_75 = (target_len/4)*3
    
    val_at_75 = target_sorted[target_len_75] #found the value at index of 75%
    
    for value in features.index: 
        if(target[value] >= val_at_75):
            target[value] = 1
        else:
            target[value] = 0
    
    #end for-loop
    features_sc = std_scalar.fit_transform(features)
    
    X_train, X_test, y_train, y_test = train_test_split(features_sc, target, test_size = 0.3, random_state=1)
    
    #end splitting into training data
    km_test_list = []
    km_test_labels = []
    km_test_scores = []
    
    for val in range(2, 6):
        
        km = KMeans(val) #applied PCA earlier to come up with 2
        km.fit(X_train, y_train)
    
        km_centers = km.cluster_centers_ #these are our centroids
        #km_list_of_centers.append(km_centers)
        km_test_list.append(km_centers)
        #print "km_centers for %s" % name_of_lists[count]
        #print km_centers
    
        km_labels = km.labels_
        km_test_labels.append(km_labels)
        #km_list_of_labels.append(km_labels)
        #print "km_labels for %s" % name_of_lists[count]
        #print km_labels
    
        s_score = silhouette_score(X_train,km_labels,metric='euclidean')
        km_test_scores.append(s_score)
        #km_list_of_s_scores.append(s_score)
    
        print "s_score for %s" % val
        print s_score
    
    print "Plot for %s" % name_of_lists[count]
    
    x_axis = range(2,6)
    
    plt.plot(x_axis, km_test_scores)  #x should be a range from 2 to 6
    #plt.ylabel('s_score')
    plt.show()
    
    km_list_of_centers.append(km_test_list)
    km_list_of_labels.append(km_test_labels)
    km_list_of_s_scores.append(km_test_scores)
    
    
    
    ### other reporting / excluding due to exploration
    
    cvs = cross_val_score(km, X_train, y_train).mean()
    km_list_of_cvs.append(cvs)
    print "Cross Val Scores for %s" % name_of_lists[count]
    print cvs
#    
    km_pred = km.predict(X_test)
    cm = confusion_matrix(y_test, km_pred)
    km_list_of_cm.append(cm)
    print "Confusion Matrix Below for %s" % name_of_lists[count]
    print cm
#    
    cr = classification_report(y_test, km_pred)
    km_list_of_cr.append(cr)
    print "Classification Report Below for %s" % name_of_lists[count]
    print classification_report(y_test, km_pred)
    
    count = count + 1
    print count

NameError: name 'mash_life' is not defined